<a href="https://colab.research.google.com/github/Degananda264/Amazon-reviews-analysis/blob/master/sentimental_analysis_using_pretrained_weights_from_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Import Libraries
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import RegexpTokenizer
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:

#Create dataframe and store the data from IMDB_Dataset.csv
data = pd.DataFrame()
data = pd.read_csv('reviews.csv', encoding='utf-8')
data.head()

,Unnamed: 0,text,sentiment
0,0,got them fast. i burned about six discs right ...,1
1,1,"Well, it fits my nano 4 quite well. And the pr...",1
2,2,"The battery arrived on the promised date, and ...",1
3,3,"Bought this one from Amazon, sold by the MiniS...",1
4,4,"So i ordered 3 chargers and surprisingly, they...",0


In [5]:
data=data.drop('Unnamed: 0',axis=1)

In [6]:
data.shape

(104975, 2)

In [7]:
data.head()

,text,sentiment
0,got them fast. i burned about six discs right ...,1
1,"Well, it fits my nano 4 quite well. And the pr...",1
2,"The battery arrived on the promised date, and ...",1
3,"Bought this one from Amazon, sold by the MiniS...",1
4,"So i ordered 3 chargers and surprisingly, they...",0


In [8]:
data[data['text'].isnull()==True]


,text,sentiment
77812,NaN,1


In [9]:
data=data.drop(77812)

In [10]:
data[data['text'].isnull()==True]

,text,sentiment


In [11]:
data=data.reset_index()

In [12]:
data.drop('index',axis=1,inplace=True)

In [13]:
data.head()

,text,sentiment
0,got them fast. i burned about six discs right ...,1
1,"Well, it fits my nano 4 quite well. And the pr...",1
2,"The battery arrived on the promised date, and ...",1
3,"Bought this one from Amazon, sold by the MiniS...",1
4,"So i ordered 3 chargers and surprisingly, they...",0


In [14]:
  
#create empty list
review_data_list = list()

indv_lines = data['text'].values.tolist()
for line in indv_lines:

	#create word tokens as well as remove puntuation in one go
	rem_tok_punc = RegexpTokenizer(r'\w+')

	tokens = rem_tok_punc.tokenize(str(line))


	#convert the words to lower case
	words = [w.lower() for w in tokens]

	#Invoke all the english stopwords
	stop_word_list = set(stopwords.words('english'))

	#Remove stop words
	words = [w for w in words if not w in stop_word_list]

	#Append words in the review_data_list list.
	review_data_list.append(words)
len(review_data_list)

104974

In [15]:

#Train a Word2Vec model using Gensim
import gensim
Embedding_Dim = 100
#train word2vec model
model = gensim.models.Word2Vec(sentences = review_data_list, size = Embedding_Dim, workers = 4, min_count = 1)
#Vocabulary size
words = list(model.wv.vocab)
print('Here is the Vocabulary Size.. %d' % len(words))

Here is the Vocabulary Size.. 56136


In [16]:
#Finding similar words
model.wv.most_similar('amazing')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('awesome', 0.8496578335762024),
 ('fantastic', 0.8327206373214722),
 ('incredible', 0.7955472469329834),
 ('wonderful', 0.7175741195678711),
 ('unbelievable', 0.717313289642334),
 ('amazed', 0.704282283782959),
 ('terrific', 0.7039431929588318),
 ('outstanding', 0.6946271657943726),
 ('great', 0.693986177444458),
 ('impressive', 0.6926626563072205)]

In [17]:
model.wv.most_similar('awful')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('horrible', 0.8600413799285889),
 ('terrible', 0.859093964099884),
 ('disappointing', 0.7036134600639343),
 ('joke', 0.6877796649932861),
 ('alright', 0.6759347915649414),
 ('poor', 0.6679002642631531),
 ('worthless', 0.665691614151001),
 ('crappy', 0.665337324142456),
 ('sucks', 0.6562924385070801),
 ('lousy', 0.655255913734436)]

In [18]:
#Finding the odd word out from the list of words given
print(model.wv.doesnt_match("man woman car".split()))

car


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
# Importing bokeh libraries for showing how words of similar context are grouped together
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

#Defining the chart
output_notebook()
plot_chart = bp.figure(plot_width=700, plot_height=600, title="A map/plot of 5000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

#Extracting the list of word vectors, limiting to 5000, each is of 200 dimensions
word_vectors = [model[w] for w in list(model.wv.vocab.keys())[:5000]]

# Reducing dimensionality by converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# Storing data in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(model.wv.vocab.keys())[:5000]

# Corresponding word appears when you hover on the data point.
plot_chart.scatter(x='x', y='y', source=tsne_df)
hover = plot_chart.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_chart)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.052s...
[t-SNE] Computed neighbors for 5000 samples in 5.099s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.174640
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.293144
[t-SNE] KL divergence after 1000 iterations: 2.244368


In [20]:
#Save word embedding model
model_file = 'amazon_reviews_word2vec_embedding.txt'
model.wv.save_word2vec_format(model_file, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Creating and trained embedding layer with pretrained weights**

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(data['text'],data['sentiment'],test_size=0.20)

In [23]:
#train_test split
x_tr, y_tr = X_train.values, Y_train.values
x_val, y_val = X_test.values, Y_test.values

In [35]:
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))
pickle.dump({'tokens':tokenizer},open( "/content/tokens_data", "wb" ))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100)
x_val_seq = pad_sequences(x_val_seq, maxlen=100)

In [25]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

53579


In [26]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(model_file)

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 56137 word vectors.


In [27]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [29]:

#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=100,trainable=False)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          5357900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          117248    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,483,469
Trainable params: 125,569
Non-trainable params: 5,357,900
_________________________________________________________________
None


In [30]:
history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc])

Train on 83979 samples, validate on 20995 samples
Epoch 1/10
83979/83979 [==============================] - 209s 2ms/step - loss: 0.3675 - acc: 0.8368 - val_loss: 0.3255 - val_acc: 0.8596

Epoch 00001: val_acc improved from -inf to 0.85959, saving model to best_model.h5
Epoch 2/10
83979/83979 [==============================] - 197s 2ms/step - loss: 0.3239 - acc: 0.8602 - val_loss: 0.3094 - val_acc: 0.8686

Epoch 00002: val_acc improved from 0.85959 to 0.86864, saving model to best_model.h5
Epoch 3/10
83979/83979 [==============================] - 198s 2ms/step - loss: 0.3027 - acc: 0.8701 - val_loss: 0.3216 - val_acc: 0.8592

Epoch 00003: val_acc did not improve from 0.86864
Epoch 4/10
83979/83979 [==============================] - 198s 2ms/step - loss: 0.2894 - acc: 0.8768 - val_loss: 0.2963 - val_acc: 0.8754

Epoch 00004: val_acc improved from 0.86864 to 0.87545, saving model to best_model.h5
Epoch 5/10
83979/83979 [==============================] - 198s 2ms/step - loss: 0.2791 - acc

In [31]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_val, batch_size=128)
print(val_acc)

20995/20995 [==============================] - 4s 182us/step
0.8776851892471313


In [48]:
pred_text="worst product."
pred_data=pd.Series(pred_text)
x_tr=pred_data.values
max_words = size_of_vocabulary
max_len = 100

In [49]:
data = pickle.load( open( "tokens_data", "rb" ) )
tokens=data['tokens']

In [50]:
x_tr_seq  = tokens.texts_to_sequences(x_tr) 

In [51]:
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100)

In [52]:
model.predict(x_tr_seq)

array([[0.001227]], dtype=float32)